# Implementation of Tausworthe, L'Ecuyer and Mersenne Twister Pseudo-Random Number generators

The project implements Tausworthe, L'Ecuyer and the Mersenne Twister pseudo-random number generators, and perform statistical tests to determine the efficiency of these generators.

## Libraries

In [ ]:
# CELL 0
# Import libraries
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import time
from datetime import timedelta
import scipy.stats as st
from scipy.stats import chi2, ksone
from tqdm import tqdm #!pip install tqdm

# Version Check
# import plotly
# import scipy
# print(f"NumPy version: {np.__version__}")
# print(f"Plotly version: {plotly.__version__}")
# print(f"SciPy version: {scipy.__version__}")
# !pip show tqdm
# !jupyter --version

## Tausworthe Generator

In [ ]:
# CELL 1
class Tausworthe:
    def __init__(self, class_parameters):
        assert isinstance(class_parameters['seed'], str), f'Provide a seed that is in string format'
        assert self.is_binary(class_parameters['seed']), f'Provide a seed that is binary and in string format'
        assert isinstance(class_parameters['r'], int), f'The Secondary Lag parameter r must be an integer greater than 0. A good choice would be to select r such that gcd(r, q) = 1, that is r and q are relatively prime. For example 53, 61, 67, 71, 73'
        assert isinstance(class_parameters['q'], int), f'The Lag parameter q must be an integer greater than 0. A good choice is to select prime numbers such that gcd(r, q) = 1 to ensure a long period. Examples: 97, 103, 127, 131, 149'
        assert len(str(class_parameters['seed'])) > class_parameters['r'] and len(str(class_parameters['seed'])) >= class_parameters['q'], f'The Lag parameters, r and q should be shorter than length of seed'
        assert math.gcd(class_parameters['r'], class_parameters['q']) == 1, f'Ensure r and q are relatively prime to ensure full period'
        assert 0 < class_parameters['r'] < class_parameters['q'], f'The length of the Lag parameters should be 0 < r < q'
        assert isinstance(class_parameters['l'], int), f'The Bit Grouping parameter l should be an integer'
        self.seed = np.array(list(class_parameters['seed']), dtype = np.int32)
        self.r = class_parameters['r']
        self.q = class_parameters['q']
        self.l = class_parameters['l']

    # staticmethod to check if seed is binary
    @staticmethod
    def is_binary(seed):
        return all(char in '01' for char in str(seed))
        
    def get_unif(self, n, disable_tqdm = False, enable_print = True):
        st = time.time()
        # Calculate the number of bits to add to the seed:
        bits_to_add = (n * self.l) - len(self.seed)

        # Create a new numpy array to store the generated bits after the XOR operation.
        # The new numpy array is intialized as the seed and an array of zeros of length (bits_to_add), concatenated.
        # The dtype for the zeros array was set as 'object' to avoid the issue of overflow
        bins_array = np.concatenate([self.seed, np.zeros(bits_to_add, dtype = object)])

        # Loop through the bins_array and perform the XOR operation.
        # The range for the loop starts at the end of the seed
        for i in tqdm(range(len(self.seed), len(bins_array)), desc = f'Generating bits', disable = disable_tqdm):
            bins_array[i] = bins_array[i - self.r] ^ bins_array[i - self.q]

        # Reshape the bins_array with the number of PRNs to be generated and the bit grouping parameter.
        # That is from 1D of shape (n * l + len(seed),) to 2D of shape (n, l)
        bins_array = bins_array.reshape(n, self.l)

        # Convert the bins_array (which now contains n binary values) into decimals
        # ref: https://stackoverflow.com/questions/15505514/binary-numpy-array-to-list-of-integers
        # dtype of object was used to avoid the issue of overflow as I have faced with large n and l values
        num_array = bins_array.dot(1 << np.arange(bins_array.shape[-1] - 1, -1, -1, dtype = object))

        # Convert the num_array (which contains n decimal values) into Unif(0, 1)
        # Divide by np.float32(2 ** l) to avoid issue of floating-point precision issues as I have faced before
        unif_array = num_array / np.float32(2 ** self.l)

        et = time.time()
        t = timedelta(seconds = et - st)
        
        if enable_print:
            print(f'-------------------------')
            print(f'Time taken to generate {n} Unif(0, 1) random numbers: {t}')
            print(f'Random Number generation rate: {t/n} /s')
            print(f'-------------------------')

        return unif_array.astype(float)

In [ ]:
# CELL 2
# Let's supply the class Tausworthe with an input.
# A good seed should be binary and not all zeros. The Lag parameters r and q should be relatively prime to
# ensure a full period. The Bit Grouping parameter l should be large enough to get high-precision uniform values.

seed = f'1011001101010100111000110100011101010111100110110101101011001011101011010111011001011100110101111101'
r = 53
q = 97
l = 53
T_class_parameters = {'seed' : seed, 'r' : r, 'q' : q, 'l' : l}
t1 = Tausworthe(T_class_parameters)

In [ ]:
# CELL 3
# Let's generate 1_000,000 Unif(0,1) random numbers

t1_unif = t1.get_unif(n = 1_000_000)

In [ ]:
# CELL 4
#Sample of numbers generated:
t1_unif[0:10]

## L'Ecuyer Generator

In [ ]:
# CELL 5
class MRG32k3a:

    #Constants to be used in the generator
    a1 = [0, 1_403_580, -810_728]
    a2 = [527_612, 0, -1_370_589]
    m1, m2 = (2 ** 32) - 209, (2 ** 32)  - 22_853
    
    def __init__(self, class_parameters):
        assert all(s < MRG32k3a.m1 for s in class_parameters['s1']), f'Seed S1 must be less than {MRG32k3a.m1} and not all zero'
        assert all(s < MRG32k3a.m2 for s in class_parameters['s2']), f'Seed S2 must be less than {MRG32k3a.m2} and not all zero'
        self.s1 = np.array(class_parameters['s1'])
        self.s2 = np.array(class_parameters['s2'])
        
     
    # function to generate the period of the L'Ecuyer generator   
    def period(self):
        p = ((MRG32k3a.m1 ** 3 - 1) * (MRG32k3a.m2 ** 3 - 1)) / 2
        print(f'The period for the MRG32k3a generator is {p}')

    def get_unif(self, n, disable_tqdm = False, enable_print = True):
        # Define X_1i and X_2i as arrays which will store the numbers generated from the linear congruential operation
        # The length of the arrays was set to length of initial seed plus number of random numbers to 
        # be generated as defined by user. This was done using np.concatenate
        X_1i = np.concatenate([self.s1, np.zeros(n, dtype = object)])
        X_2i = np.concatenate([self.s2, np.zeros(n, dtype = object)])
        
        # Create an np.zeros array to store the unif numbers generated
        unif = np.zeros(n, dtype = object)

        st = time.time()
        # Define a loop to run from range(3, n + 3). Starting range 3 was chosen to access the third position from
        # the X_1i and X_2i arrays.
        for i in tqdm(range(3, n + 3), desc = f'Progress', disable = disable_tqdm):
            X_1i[i] = sum(a1i * x1 for a1i, x1 in zip(MRG32k3a.a1[::-1], X_1i[i - 3:i:1])) % MRG32k3a.m1
            X_2i[i] = sum(a2i * x2 for a2i, x2 in zip(MRG32k3a.a2[::-1], X_2i[i - 3:i:1])) % MRG32k3a.m2
            
            unif[i-3] = ((X_1i[i] - X_2i[i]) % MRG32k3a.m1) / (MRG32k3a.m1) 
        et = time.time()
        t = timedelta(seconds = et - st)
        if enable_print:
            print(f'-------------------------')
            print(f'Time taken to generate {n} Unif(0, 1) random numbers: {t}')
            print(f'Random Number generation rate: {t/n} /s')
            print(f'-------------------------')
        return unif.astype(float)


In [ ]:
# CELL 6
# Provide two vectors (s10, s11, s12) and (s20, s21, s22) as seeds to the generator. 
# Before the procedure is called for the first time, one must initialize s10, s11, s12 to
# (exact) non-negative integers less than m1 and not all zero, and s20, s21, s22 to non-negative integers less than m2 and not all equal to zero. Seeds with good figure of Merit M32 are provided below. The figure of Merit M32 for the seeds below is 0.63359

s1 = [16807, 28247529, 1622650073]
s2 = [984943658, 1144108930, 470211272]

E_class_parameters = {'s1' : s1, 's2' : s2}

In [ ]:
# CELL 7
# Create an instance of the generator

e1 = MRG32k3a(E_class_parameters)

In [ ]:
# CELL 8
# Generate 1_000,000 Unif(0, 1) random numbers

e1_unif = e1.get_unif(1_000_000)

In [ ]:
# CELL 9
# Sample of numbers generated:
e1_unif[0:10]

## Mersenne Twister Generator

In [ ]:
# CELL 10
class MT19937:
    # Parameters
    w, N, M, r = 32, 624, 397, 31
    MATRIX_A = 0x9908b0df
    u, s, t, l = 11, 7, 15, 18 # Tempering parameters
    d = 0xffffffff # 
    f = 1812433253 # See Knuth TAOCP Vol2. 3rd Ed. P.106 for multiplier.
    TEMPERING_MASK_B = 0x9d2c5680 # Tempering mask
    TEMPERING_MASK_C = 0xefc60000 # Tempering mask
    UPPER_MASK = 0x80000000 # To extract the higher (w - r) significant bits
    LOWER_MASK = 0x7fffffff # To extract the lower r significant bits for the next number
    mag01 = np.array([0x0, MATRIX_A]) # mag01[x] = x * MATRIX_A for x = {0, 1}
    

    def __init__(self, class_parameters):
        self.mti = MT19937.N # A counter to keep track everytime a number is generated 
        
        # set up array for state vectors
        self.mt = np.zeros(MT19937.N, dtype = np.uint32)
        
        # use the seed as the first state. AND operation to ensure the seed is 32-bit
        self.mt[0] = np.uint32(class_parameters['seed'] & MT19937.d)

        # update the state vector using a simple Linear Congruential method.
        # Note the use of various np.uintxx. NumPy enforces strict 32-bit integerlimits.
        # In order to avoid Runtime Overflow error caused by large numbers, I converted values
        # to uint64 and back to uint32
        for i in range(1, MT19937.N):
            self.mt[i] = np.uint32(np.uint64(MT19937.f) * np.uint64(self.mt[i - 1] ^ (self.mt[i - 1] >> (MT19937.w - 2))) + np.uint64(i))
            
            

    def twist(self):
        # This is equivalent to getting the most significant bit from mt[i] and concatenating to the least significant bit
        # of the mt[i+1]. The np.roll allows to loop around to the beginning of the array. Next is the XOR operation of three figures.
        # The first is the state vector starting at position 397 (achieved using np.roll), next is a bitwise right shift of y,
        # and then its the mag01 which is either 0 or MATRIX_A. The mti counter is reset to zero.
        y = np.add((self.mt & MT19937.UPPER_MASK).astype(np.uint32), (np.roll(self.mt, -1) & MT19937.LOWER_MASK).astype(np.uint32))
        self.mt = np.roll(self.mt, -self.M) ^ np.bitwise_right_shift(y, 1) ^ MT19937.mag01[y & 0x1]
        self.mti = 0

    def temper(self):
        # Tempering checks and performs twisting if it has not been done.
        if self.mti >= MT19937.N:
            self.twist()

        # Set up an intermediate variable that performs tempering as described below
        # for each of the state vectors
        y = self.mt[self.mti]
        self.mti += 1

        # Tempering steps which includes various bitwise shifts
        # TEMPERING_SHIFT_U(y)
        y ^= np.bitwise_right_shift(y, MT19937.u)
        # TEMPERING_SHIFT_S(y)
        y ^= np.bitwise_left_shift(y, MT19937.s) & MT19937.TEMPERING_MASK_B
        # TEMPERING_SHIFT_T(y)
        y ^= np.bitwise_left_shift(y, MT19937.t) & MT19937.TEMPERING_MASK_C
        # TEMPERING_SHIFT_L(y)
        y ^= np.bitwise_right_shift(y, MT19937.l)

        return y / (2 ** MT19937.w - 1)

    
    def get_unif(self, n: int, enable_print = True, disable_tqdm = False):
        # Set up an initial array of zeros which will be filled with the random numbers
        unif = np.zeros(n)
        st = time.time()
        for i in tqdm(range(n), desc = f'Generating random numbers:', disable = disable_tqdm):
            unif[i] = self.temper()
        et = time.time()
        t = timedelta(seconds = et - st)
        if enable_print:
            print(f'-------------------------')
            print(f'Time taken to generate {n} Unif(0, 1) random numbers: {t}')
            print(f'Random Number generation rate: {t/n} /s')
            print(f'-------------------------')
            
        return unif

In [ ]:
# CELL 11
# # An example. Create an instance of the generator. First define the class parameters, which in this case is just the seed.

M_class_parameters = {'seed' : 4357}

mt1 = MT19937(M_class_parameters)

In [ ]:
# CELL 12
# Generate 1_000,000 Unif(0, 1) random numbers

mt1_unif = mt1.get_unif(n = 1_000_000)

In [ ]:
# CELL 13
# Sample of numbers generated:

mt1_unif[0:10]

## Spatial Distribution

In [ ]:
# CELL 14
# Let's create a list of dictionaries called gen_list that will store the name, the generated Unif(0,1) distributions and a color speciffic to the distribution.
# This list serves as a single variable (gen_list) to perform the subsequent tests in a streamlined manner, allowing for concise one-line operations and plot generation.

gen_list = [
    {'name' : 'Tausworthe', 'unif' : t1_unif, 'color' : '#B3A369'},
    {'name' : 'L\'Ecuyer', 'unif' : e1_unif, 'color' : '#003057'},
    {'name' : 'Mersenne Twister', 'unif' : mt1_unif, 'color' : '#FFFFFF'}
             ]

In [ ]:
# CELL 15
# Function defined below, hist() takes in the random numbers generated from the three generators and plots the histograms.

def hist(unifs):
    # Add an assertion to check if all the distributions have the same length.
    assert len({len(unif['unif']) for unif in unifs}) == 1, 'For better visualization, enter uniforms of same length'
    
    # n_bins will determine the bin size of the histograms
    n_bins = 25
    
    # Create subplots grid of 1 row and 3 columns
    fig = make_subplots(rows = 1, cols = len(unifs), subplot_titles = [unif['name'] for unif in unifs])
    
    # Add histogram traces
    for i, gen in enumerate(unifs):
      fig.add_trace(go.Histogram(x = gen['unif'], 
                                     xbins = dict(start = 0, end = 1, size = 1/n_bins),
                                     marker = dict(line = dict(color = 'black', width = 2), color = gen['color']),
                                     hovertemplate = "<br>".join([f'{gen['name']}', "Range: %{x}", "Frequency of Random Numbers: %{y}"]),
                                     name = ''), row = 1, col = i + 1)
      fig.add_hline(y = len(gen['unif']) / n_bins, 
                   line_width = 2, 
                   line_dash = "dash", 
                   line_color = "red")
      fig.update_xaxes(title_text="Value", title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = i + 1)
      fig.update_yaxes(title_text="Frequency", title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = i + 1)
    
    # Update layout for better visualization
    fig.update_layout(title = {'text' : '<b>Histograms of Generated Sample Distributions</b>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'textcase' : 'upper',
                                          'size' : 25}
                                          },
                    width = 2000,
                    height = 600,
                    showlegend = False,
                    paper_bgcolor = 'white',
                    plot_bgcolor = '#777777'
                  )
    fig.show() 

In [ ]:
# CELL 16
# This block will display the histograms. The y-axis represents the frequency (or count) of the samples falling in the ranges as described by the x-axis.
hist(gen_list)

In [ ]:
# CELL 17
# The function below will draw a scatter plot of adjacent random numbers.

def scatter(unifs):
    # Add an assertion to check if all the distributions have the same length.
    assert len({len(unif['unif']) for unif in unifs}) == 1, 'For better visualization, enter uniforms of same length'
    
    # Create subplots grid of 1 row and 3 columns
    fig = make_subplots(rows = 1, cols = len(unifs), subplot_titles = [unif['name'] for unif in unifs])
    
    # Add scatter traces
    for i, gen in enumerate(unifs):
      fig.add_trace(go.Scatter(x = gen['unif'][0:999],
                                y = gen['unif'][1:1000],
                                mode = 'markers',
                                marker = dict(line = dict(color ='black', width=1), color = gen['color']),
                                hovertemplate = "<br>".join([f'{gen['name']}', 'X<sub>i</sub> : %{x}', 'X<sub>i+1</sub> : %{y}']), name = ''
                              ), row = 1, col = i + 1)
      
      fig.update_xaxes(title_text='X<sub>i</sub>', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = i + 1)
      fig.update_yaxes(title_text='X<sub>i+1</sub>', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = i + 1)
    
    # Update layout for better visualization
    fig.update_layout(title = {'text' : '<b>2-D Scatter Plots</b>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'textcase' : 'upper',
                                          'size' : 25}
                                          },
                    width = 2000,
                    height = 600,
                    showlegend = False,
                    paper_bgcolor = 'white',
                    plot_bgcolor = '#777777'
                  )
    fig.show()

In [ ]:
# CELL 18
# This block will display the 2-dimensional scatter plot.
scatter(gen_list)

In [ ]:
# CELL 19
# The function below will draw a 3d-scatter plot of adjacent random numbers.

def scatter3d(unifs):
    # Add an assertion to check if all the distributions have the same length.
    assert len({len(unif['unif']) for unif in unifs}) == 1, 'For better visualization, enter uniforms of same length'
    
    # Create subplots grid of 1 row and 3 columns
    fig = make_subplots(rows = 1, cols = len(unifs), specs=[[{'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}]], subplot_titles = [unif['name'] for unif in unifs])
    
    # Define scene settings
    scene_settings = {'xaxis' : {'title' : 'X<sub>i</sub>', 'tickfont' : {'size' : 15},'title_font' : {'size' : 15}, 'backgroundcolor' : '#777777', 'gridwidth' : 1},
                      'yaxis' : {'title' : 'X<sub>i+1</sub>', 'tickfont' : {'size' : 15}, 'title_font' : {'size' : 15}, 'backgroundcolor' : '#777777', 'gridwidth' : 1},
                      'zaxis' : {'title' : 'X<sub>i+2</sub>','tickfont' : {'size' : 15},'title_font' : {'size' : 15}, 'backgroundcolor' : '#777777', 'gridwidth' : 1},
                      'camera' : {'eye' : {'x' : 2.0, 'y' : 2.0, 'z' : 2.0}}
                      }
    
    # Add scatter traces
    for i, gen in enumerate(unifs):
      fig.add_trace(go.Scatter3d(x = gen['unif'][0:999],
                                y = gen['unif'][1:1000],
                                z = gen['unif'][2:1001],
                                mode = 'markers',
                                marker = {'line' : {'color' : 'black', 'width' : 1}, 'size' : 4, 'color' : gen['color']},
                                hovertemplate = "<br>".join([f'{gen['name']}', "X<sub>i</sub> : %{x}", "X<sub>i+1</sub> : %{y}", "X<sub>i+2</sub> : %{z}"]),
                                name = gen['name'], hoverlabel={'bgcolorsrc' : 'blue'}), row = 1, col = i + 1)

    # Update layout for better visualization
    fig.update_layout(title = {'text' : '<b>3-D Scatter Plots</b>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'textcase' : 'upper',
                                          'size' : 25}
                                          },
                      scene = scene_settings,
                      scene2 = scene_settings,
                      scene3 = scene_settings,
                      width = 2000,
                      height = 800,
                      showlegend = True,
                      legend=dict(bgcolor='lightgray'),
                      margin = {'l' : 0, 'r' : 0, 't' : 0, 'b' : 0}
                    )
    fig.show()

In [ ]:
# CELL 20
# This block will display the 3d-scatter plot.
scatter3d(gen_list)

## Serial-Correlation Test

Serial correlation, or autocorrelation occurs when a variable and a lagged version of itself are observed to be correlated over periods of time.  It quantifies the similarity (or dissimilarity) between observations of a random variable at different points in time. It is an essential mathematical tool for identifying repeating patterns or hidden periodicities. Serial Correlation is widely used in signal processing, time domain and time series analysis (Ref: https://en.wikipedia.org/wiki/Autocorrelation).

The most common way of measuring a linear correlation is by computing the Pearson correlation coefficient ($\rho$).

$$
\rho = \frac{Cov(R_i, R_{i+k})}{\sqrt{Var(R_i)}\sqrt{Var(R_{i+k})}}
$$

The Pearson coefficient, $\rho$ lies between -1 and +1 that measures the strength and direction of the relationship between two variables. In the equation above, the coefficient is measuring the correlation between the $\textit{i}$-th number and the ($\textit{i+k}$)-th. When k = 1, we call it the lag-1 correlation of $R_i$'s. 

Note that uncorrelated does not necessarily mean random. Data that has significant correlation is not random. However, data that does not show significant correlation can still exhibit non-randomness in other ways. Correlation is just one measure of randomness.

The above equation can be slightly modified since we are testing for correlation of a Unif(0,1) distribution. Lets suppose we are interested in the correlation coefficient between consecutive random numbers from a sample distribution with lag k. Then we represent the samples as:
$$
X = \{R_1, R_2, ... , R_{n-k}\}\\
Y = \{R_{k+1}, R_{k+2}, ... , R_{n-k}, R_{n-k+1}, ..., R_n\}
$$

The coefficient can be re-written as:

$$
\rho = \frac{(E[X - E[X]))(E[Y - E[Y]])}{\sqrt{Var(X)}\sqrt{Var(Y)}}
$$
where we have substituted the definition of Cov(X, Y). And for a Unif(0,1) distribution, the expected value and the variance for all i's are:
$$
E[R_i] = \frac{1}{2}
$$
$$
Var(R_i) = \frac{1}{12}
$$
Substituting these equations, we obtain the following:
$$
\rho = \frac{(E[X - E[X]))(E[Y - E[Y]])}{\sqrt{Var(X)}\sqrt{Var(Y)}}
$$
$$
or, \rho = \frac{E[XY] - E[X]E[Y]}{\sqrt{Var(X)}\sqrt{Var(Y)}}
$$
$$
or, \rho = \frac{E[XY] - \frac{1}{2}.\frac{1}{2}}{\sqrt{\frac{1}{12}}\sqrt{\frac{1}{12}}}
$$
$$
or, \rho = \frac{E[XY]}{\sqrt{\frac{1}{12}}\sqrt{\frac{1}{12}}} - \frac{\frac{1}{2}.\frac{1}{2}}{\sqrt{\frac{1}{12}}\sqrt{\frac{1}{12}}}
$$
$$
or, \rho = 12E[XY] - 3
$$
Now lets evaluate $E[XY]$,
$$
E[XY] = \frac{1}{n-k}\sum_{i = 1}^{n-k}R_iR_{i+k}
$$
Thus, now we can write the final equation as
$$
\rho = \frac{12}{n-k}\sum_{i = 1}^{n-k}R_iR_{i+k} - 3
$$



In [ ]:
# CELL 21
def correlation(unifs, lag = 500):
    # Add an assertion to check if all the distributions have the same length.
    assert len({len(unif['unif']) for unif in unifs}) == 1, 'For better visualization, enter uniforms of same length'
    
    # For better visualization, let's set the max lag to be 500
    assert lag <= 500, f'Max lag should be less than 500'
    
    # length of the distribution
    n = len(unifs[0]['unif'])
    
    # Make sure that the lag is lesser than the length of the distribution
    assert lag < n, f'Value of Lag should be less than length of the uniform distribution array'

    # Create subplots grid of 1 row and 3 columns
    fig = make_subplots(rows = 1, cols = len(unifs), subplot_titles = [unif['name'] for unif in unifs])
    
    # Variable 'corr' will store the correlation coefficients at different lags for the three distributions
    # generated from the PRNGs 
    for i, unif in enumerate(unifs):
        corr = np.zeros(shape = lag)
        for j in range(1, lag + 1):
            X = unif['unif'][:n-j]
            Y = unif['unif'][j:]
            XY = np.sum(np.multiply(X, Y))
            corr[j-1] = (12 * XY) / (n - j) - 3

        
        fig.add_trace(go.Scatter(x = np.arange(start = 1, stop = lag),
                                 y = corr,
                                 mode = 'lines',
                                 marker = dict(color = unif['color']),
                                 hovertemplate = "<br>".join([f'{unif['name']}', 'Corr : %{y}', 'Lag : %{x}']),
                                 name = ''), row = 1, col = i + 1)
        fig.add_hline(y = 0, line_width = 2, line_color = 'red')
        fig.update_xaxes(title_text='Lag', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = i + 1)
        fig.update_yaxes(title_text='Correlation Coefficient', title_font_size = 20, tickfont_size = 15, range = [-max(corr) - 0.01, max(corr) + 0.01], gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = i + 1)
        
        
    fig.update_layout(title = {'text' : '<b>Serial Correlation Test</b>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'textcase' : 'upper',
                                          'size' : 25}
                                          },
                    width = 2000,
                    height = 600,
                    showlegend = False,
                    paper_bgcolor = 'white',
                    plot_bgcolor = '#777777'
                  )
    fig.show()

In [ ]:
# CEL  22
# Display the line plot of correlation cofficient vs lag
correlation(gen_list, 500)

## Uniformity Tests

### Chi-squared Goodness-of-Fit test

For each generator, I have generated 1_000,000 random numbers. 

In [ ]:
# CELL 23
def gof(gen_list, gen, bins, alpha):
    # Get the required distribution
    for item in gen_list:
        if item['name'] == gen:
            unif = item['unif']
            nameU = item['name']
            colorU = item['color']

    # Print the hypothesis
    print(f'Testing Goodness-of-Fit for the {nameU} PRN Generator')
    print(f'\033[4mHypothesis:\033[0m')
    print(f'H\u2080: The random numbers are identically distributed')
    print(f'H\u2081: The random numbers are not identically distributed')
    print(f'-----------------------')

    # Count the number of random number in 'k' bins
    unif_counts, unif_bin_edges = np.histogram(unif, bins = bins, range = (0.0, 1.0))

    # Get the intervals
    intervals = [(i/bins, (i+1)/bins) for i in range(bins)]
    print(f'The intervals for the range (0, 1) divided into {bins} sections:')
    print(intervals)

    # Calculate the expected value for each bin, E
    E = int(len(unif) / bins)
    print(f'The expected number of random numbers in each bin is {E}')

    print(f'The observed number of random numbers in each intervals: {unif_counts}')

    # Claculate the chi-squared statistic
    chi_squared_stat = sum((unif_counts - E) ** 2 / E)
    print(f'The chi_squared statistic = {chi_squared_stat}')

    # Calculate the chi_squared value at the desired significance level
    df = bins - 1 # df = degree of freedom
    chi_squared = chi2.ppf(1 - alpha, df)
    print(f'The critical chi-squared value at confidence level {(1 - alpha) * 100}% = {chi_squared}')

    if chi_squared_stat <= chi_squared:
        print(f'We see that {chi_squared_stat} < {chi_squared}. Thus,')
        print(f'----> \033[1mWe fail to reject the null hypothesis H\u2080\033[0m <----')
        print(f'The sequence of numbers are in fact identically distributed')
    else:
        print(f'We see that {chi_squared_stat} > {chi_squared}. Thus,')
        print(f'----> \033[1mReject the null hypothesis H\u2080\033[0m <----')
        print(f'The sequence of numbers are in fact not identically distributed')

In [ ]:
# CELL 24
# Goodness-of-Fit test for the Tausworthe generator.
# Use gof(unif, bins, alpha) to test the hypothesis H0 = The random numbers are identically distributed

gof(gen_list, 'Tausworthe', bins = 5, alpha = 0.05)

In [ ]:
# CELL 25
# Goodness-of-Fit test for the L'Ecuyer generator
# Use gof(unif, bins, alpha) to test the hypothesis H0 = The random numbers are identically distributed

gof(gen_list, 'L\'Ecuyer', bins = 5, alpha = 0.05)

In [ ]:
# CELL 26
# Goodness-of-Fit test for Mersenne Twister generator
# Use gof(unif, bins, alpha) to test the hypothesis H0 = The random numbers are identically distributed

gof(gen_list, 'Mersenne Twister', bins = 5, alpha = 0.05)

### Kolmogórov-Smirnov Test

In [ ]:
# CELL 27
def ks(gen_list, gen, alpha):
    # Get the distribution
    for item in gen_list:
        if item['name'] == gen:
            unif = item['unif']
            nameU = item['name']
            colorU = item['color']

    # Print the hypothesis
    print(f'Kolmogórov-Smirnov Test for {nameU} PRN Generator')
    print(f'\033[4mHypothesis:\033[0m')
    print(f'H\u2080: The random numbers are identically distributed')
    print(f'H\u2081: The random numbers are not identically distributed')
    print(f'-----------------------')

    # Sample (or distribution) size, n
    n = len(unif)
    
    # Compute indices
    indices = np.arange(1, (n + 1))
    
    # Ordered points
    print(f'Creating ordered points: ...')
    st = time.time()
    unif_ordered = np.sort(unif)
    t = timedelta(seconds = time.time() - st)
    print(f'Create ordered points: \033[1mDone\033[0m (Time taken: {t})')

    # Calculate D_plus
    D_plus = max((indices/n) - unif_ordered)
    print(f'D\u207A = {D_plus}')
    
    
    # Calculate D_minus
    D_minus = max((unif_ordered - (indices - 1) / n))
    print(f'D\u207B = {D_minus}')
    
    
    # Calcuate D = max{D_plus, D_minus}
    D = max(D_plus, D_minus)
    print(f'Thus, D = max\u007bD\u207A, D\u207B\u007d = max\u007b{D_plus}, {D_minus}\u007d = {D}')
    
    # Get the K-S critical value at the significance level
    D_ks = ksone.ppf(1 - alpha/2, n)
    print(f'The critical value of D(\u03B1,n) at significance level of {alpha} is {D_ks}')

    # Check condition for the hypothesis testing
    if D < D_ks:
      print(f'We see that D < D(\u03B1,n). Thus,')
      print(f'----> \033[1mWe fail to reject the null hypothesis H\u2080\033[0m <----')
      print(f'The sequence of numbers are in fact identically distributed')
    else:
       print(f'We see that D > D(\u03B1,n). Thus,')
       print(f'----> \033[1mReject the null hypothesis H\u2080\033[0m <----')
       print(f'The sequence of numbers are in fact not identically distributed')
       
    trace1 = go.Scatter(x = np.concatenate([np.array([0]), unif_ordered, np.array([1])],),
                        y = np.arange(len(unif_ordered) +2) / (n + 1),
                        mode = 'lines',
                        marker = dict(color = colorU),
                        hovertemplate = '<br>'.join([f'{nameU}', 'Cumulative prob: %{y}', 'X: %{x}']),
                        showlegend = False,
                        name = '')
    trace2 = go.Scatter(x = np.linspace(start = 0, stop = 1, num = n+2), 
                        y = np.linspace(start = 0, stop = 1, num = n+2), 
                        mode = 'lines', marker = dict(color = 'red'),
                        hovertemplate = '<br>'.join([f'True CDF', 'Cumulative prob: %{y}', 'X: %{x}']),
                        showlegend = False,
                        name = '')
    
    fig = make_subplots(rows = 1, cols = 2, subplot_titles = ['Empirical vs True CDF', 'Zoomed In'])
    fig.add_trace(trace1, row = 1, col = 1)
    fig.add_trace(trace2, row = 1, col = 1)
    fig.add_trace(trace1.update(name = 'Empirical CDF', showlegend = True), row = 1, col = 2)
    fig.add_trace(trace2.update(name = 'True CDF', showlegend = True), row = 1, col = 2)
    fig.update_xaxes(title_text = 'X', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1)
    fig.update_yaxes(title_text = 'Cumulative Probability', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1)
    fig.update_layout(title = {'text' : f'Comparison between the theoretical uniform CDF and the empirical CDF<br><sub>{nameU}</sub>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'textcase' : 'upper',
                                          'size' : 20}
                                          },
                    width = 1200,
                    height = 600,
                    paper_bgcolor = 'white',
                    plot_bgcolor = '#777777',
                    legend=dict(bgcolor='lightgray'))
    fig.update_xaxes(range = [0, 0.004], row = 1, col = 2)
    fig.update_yaxes(range = [0, 0.004], row = 1, col = 2)
    fig.show() 

In [ ]:
# CELL 28
# Kolmogórov-Smirnov test for Tausworthe generator
# Note: The algorithm takes time (~ 1min 30secs) depending on the length of the input array
#       The algorithm does a sorting process, and sorting is a O(nlogn) operation.

# Use ks(unif, alpha) to test the hypothesis H0: The random numbers are identically distributed
ks(gen_list, 'Tausworthe', alpha = 0.05)

In [ ]:
# CELL 29
# Kolmogórov-Smirnov test for L'Ecuyer generator
# Note: The algorithm takes time (~ 1min 30secs) depending on the length of the input array
#       The algorithm does a sorting process, and sorting is a O(nlogn) operation.

# Use ks(unif, alpha) to test the hypothesis H0: The random numbers are identically distributed

ks(gen_list, 'L\'Ecuyer', alpha = 0.05)

In [ ]:
# CELL 30
# Kolmogórov-Smirnov test for Mersenne Twister generator
# Note: The algorithm takes time (~ 1min30secs) depending on the length of the input array
#       The algorithm does a sorting process, and sorting is a O(nlogn) operation.

# Use ks(unif, alpha) to test the hypothesis H0: The random numbers are identically distributed

ks(gen_list, 'Mersenne Twister', alpha = 0.05)

## Independence Tests

### von Neumann Ratio Test

Von Neumann (1941) considered n successive observations from a normal population with mean $\mu$ and variance $\sigma^2$. Using the (slightly) biased estimator of $\sigma^2$,

$$
S^2 = \frac{1}{n} \sum_{i=1}^n (X_i - \bar{X})^2
$$

and the mean squared difference between successive observations,

$$d^2 = \frac{1}{n - 1}\sum_{i=1}^{n-1}(X_{i+1} - X_i)^2$$

von Neumann examined the ratio $V = d^2/S^2$. This ratio is intuitively appealing as a measure of independence between successive observations, because $S^2$ represents a measure of variability that does not consider the order of observations, whereas the importance of order is explicit in $d^2$.

Thus the von Neumann statistic,

 $$VN = \frac{d^2}{S^2}$$

 It is known that $(VN - \mu)/\sigma$ is asymptotically standard normal, where $\mu = \frac{2n}{n-1}$ and $\sigma^2 = \frac{4(n-2)}{n^2 -1}$



In [ ]:
# CELL 31
def vn(gen_list, gen, alpha):
    # Get the distribution
    for item in gen_list:
        if item['name'] == gen:
            unif = item['unif']
            nameU = item['name']
            colorU = item['color']
    
    # Print the hypothesis
    print(f'Von Neumann Ratio Test for {nameU} PRN Generator')
    print(f'\033[4mHypothesis:\033[0m')
    print(f'H\u2080: The random numbers are independent')
    print(f'H\u2081: The random numbers are not independent')
    print(f'-----------------------')

    # number of samples
    n = np.float64(len(unif))

    # mean-squared difference between successive observations
    d2 = np.sum(np.square(np.diff(unif))) / (n-1)
    
    # von Neumann statistic
    vn = d2 / np.var(unif)
    
    # Assuming we have generated a large number of samples
    mu = np.float64(2*n/(n-1))
    sigma2 = np.float64(4*n/(n**2 - 1))
    vn_norm = abs((vn - mu) / (sigma2 ** (0.5)))
    print(f'von Neumann statistic after normalization with \u03BC and \u03C3, Z_stat = {vn_norm}')

    # Get the z-score value from the significance level
    z_score = st.norm.ppf(1-alpha/2)
    print(f'The critical z_score at significance level {chr(945)} = {alpha}, Z({chr(945)}, n) is {z_score}')
    
    if abs(vn_norm) < z_score:
        print(f'We see that |Z_stat| < Z({chr(945)}, n). Thus,')
        print(f'----> \033[1mWe fail to reject the null hypothesis H\u2080\033[0m <----')
        print(f'The sequence of numbers are in fact independent')
    else:
        print(f'We see that |Z_stat| > Z({chr(945)}, n). Thus,')
        print(f'\033[1mReject the null hypothesis H\u2080\033[0m')
        print(f'The sequence of numbers are in fact not independent')

In [ ]:
# CELL 32
# Von Neumann Ratio test for the Tausworthe generator

# Use vn(list, name, alpha) to test the hypothesis H0: The random numbers are independent
vn(gen_list, 'Tausworthe', alpha = 0.05)

In [ ]:
# CELL 33
# Von Neumann Ratio test for the L'Ecuyer generator

# Use vn(unif, alpha) to test the hypothesis H0: The random numbers are independent
vn(gen_list, 'L\'Ecuyer', alpha = 0.05)

In [ ]:
# CELL 34
# Von Neumann Ratio test for the Mersenne Twister generator

# Use vn(unif, alpha) to test the hypothesis H0: The random numbers are independent
vn(gen_list, 'Mersenne Twister', alpha = 0.05)

### Runs Up/Down Test

In [ ]:
# CELL 35
def runsUD(gen_list, gen, alpha):
    # Get the distribution
    for item in gen_list:
        if item['name'] == gen:
            unif = item['unif']
            nameU = item['name']
            colorU = item['color']
            
    # Print the hypothesis
    print(f'Runs Up and Down Test for {nameU} PRN Generator')
    print(f'\033[4mHypothesis:\033[0m')
    print(f'H\u2080: The random numbers are independent')
    print(f'H\u2081: The random numbers are not independent')
    print(f'-----------------------')

    # number of samples
    n = len(unif)

    # algorith to get the number of runs, A
    # Logic: np.diff(unif) gives a numpy array with discrete differences along the given axis,
    # np.sign() converts the array's positive values to 1 and negative values to -1. Now compare 
    # adjacent values for inflection point, that is when 1 becomes -1 and vice-versa.
    A = np.sum((np.sign(np.diff(unif))[:-1] != np.sign(np.diff(unif))[1:]))
    print(f'The number of runs in the sample of numbers provided, A = {A}')
    print(f'-----------------------')
    
    # Get the mu and var
    mu = np.float64((2*n - 1)/3)
    var = np.float64((16*n - 29)/90)
    print(f'mu = {mu}')
    print(f'var = {var}')
    

    # Calculate the test statistic
    Z_stat = (A - mu)/(var ** 0.5)
    print(f'Z-statistic = {Z_stat}')

    # Get the critical z-score value from the significance level
    z_score = st.norm.ppf(1 - alpha / 2)
    print(f'The critical z_score at significance level {chr(945)} = {alpha}, Z({chr(945)}, n) is {z_score}')

    if abs(Z_stat) < z_score:
        print(f'We see that |Z_stat| < Z({chr(945)}, n). Thus,')
        print(f'----> \033[1mWe fail to reject the null hypothesis H\u2080\033[0m <----')
        print(f'The sequence of numbers are in fact independent')
    else:
        print(f'We see that |Z_stat| < Z({chr(945)}, n). Thus,')
        print(f'\033[1mReject the null hypothesis H\u2080\033[0m')
        print(f'The sequence of numbers are in fact not independent')

In [ ]:
# CELL 36
# Runs Up-and-Down test for Tausworthe generator

# Use runsUD(unif, alpha) to test the hypothesis H0: The random numbers are independent
runsUD(gen_list, 'Tausworthe', alpha = 0.05)

In [ ]:
# CELL 37
# Runs Up-and-Down test for L'Ecuyer generator

# Use runsUD(unif, alpha) to test the hypothesis H0: The random numbers are independent
runsUD(gen_list, 'L\'Ecuyer', alpha = 0.05)

In [ ]:
# CELL 38
# Runs Up-and-Down test for Mersenne Twister generator

# Use runsUD(unif, alpha) to test the hypothesis H0: The random numbers are independent
runsUD(gen_list, 'Mersenne Twister', alpha = 0.05)

### Runs Test "Above and Below the Mean"

In [ ]:
# CELL 39
def runsABM(gen_list, gen, alpha):
    # Get the distribution
    for item in gen_list:
        if item['name'] == gen:
            unif = item['unif']
            nameU = item['name']
            colorU = item['color']
            
    # Print the hypothesis
    print(f'Runs Above and Below the Mean Test for {nameU} PRN Generator')
    print(f'\033[4mHypothesis:\033[0m')
    print(f'H\u2080: The random numbers are independent')
    print(f'H\u2081: The random numbers are not independent')
    print(f'-----------------------')

    # number of samples
    n = np.float64(len(unif))

    # Get the number of runs, B
    # Logic: We subtract 0.5 from each element of array, and use
    # np.sign() to denote the value as 1 if it is positive and -1 if neagtive.
    # Compare adjacent values for sign change, and add 1 to account for the extra
    # comparison.
    B = np.sum(np.sign(unif - 0.5)[:-1] != np.sign(unif - 0.5)[1:]) + 1
    print(f'The number of runs in the sample of numbers provided, B = {B}')
    print(f'-----------------------')

    # Get n1, number of observations greater than equal to 0.5, and 
    # n2 = number of observations less than 0.5
    n1 = np.float64(np.sum(unif >= 0.5))
    n2 = np.float64(n - n1)
    print(f'Number of samples greater than or equal to 0.5 = {int(n1)}')
    print(f'Number of samples less than 0.5 = {int(n2)}')
    print(f'-----------------------')

    # Calculate mu and variance using n1 and n2
    mu = np.float64(((2 * n1 * n2)/ n) + 0.5)
    var = np.float64((2 * n1 * n2 * (2 * n1 * n2 - n)) / (n * n * (n - 1)))
    print(f'mu = {mu}')
    print(f'var = {var}')

    # Calculate the test statistic, Z_stat
    Z_stat = (B - mu) / (var ** 0.5)
    print(f'Z-statistic = {Z_stat}')

    # Get the critical z-score value from the significance level
    z_score = st.norm.ppf(1-alpha/2)
    print(f'The critical z_score at significance level {chr(945)} = {alpha}, Z({chr(945)}, n) is {z_score}')

    # Check the condition for the hypothesis
    if abs(Z_stat) < z_score:
        print(f'We see that |Z_stat| < Z({chr(945)}, n). Thus,')
        print(f'----> \033[1mWe fail to reject the null hypothesis H\u2080\033[0m <----')
        print(f'The sequence of numbers are in fact independent')
    else:
        print(f'We see that |Z_stat| < Z({chr(945)}, n). Thus,')
        print(f'\033[1mReject the null hypothesis H\u2080\033[0m')
        print(f'The sequence of numbers are in fact not independent')

In [ ]:
# CELL 40
# Runs Above-and-Below the Mean test for Tausworthe generator

# Use runsABM(unif, alpha) to test the hypothesis H0: The random numbers are independent
runsABM(gen_list, 'Tausworthe', alpha = 0.05)

In [ ]:
# CELL 41
# Runs Above-and-Below the Mean test for L'Ecuyer generator

# Use runsABM(unif, alpha) to test the hypothesis H0: The random numbers are independent
runsABM(gen_list, 'L\'Ecuyer', alpha = 0.05)

In [ ]:
# CELL 42
# Runs Above-and-Below the Mean test for Mersenne Twister generator

# Use runsABM(unif, alpha) to test the hypothesis H0: The random numbers are independent
runsABM(gen_list, 'Mersenne Twister', alpha = 0.05)

## Monte-Carlo Simulation for estimating the value of $\pi$

In [ ]:
# CELL 43
def MC(gen_list, gen):
    # Get the distributions
    for item in gen_list:
        if item['name'] == gen:
            unif1 = item['unif'][0:10000]
            unif2 = item['unif2'][0:10000]
            nameU = item['name']
            colorU = item['color']
    # Check whether the distributions are of the same length
    assert len(unif1) == len(unif2), 'The two samples should be of the same length'
    
    # Get the length of the distribution
    n = len(unif1)
    # Create an array of integers. This will be used in subplot number 2
    indices = np.arange(1, n + 1)
    
    # Pairing corresponding elements from the two distributions as cordinates in a unit square, and calculating
    # the distance of the points from the center (0.5, 0.5)
    dist = np.sqrt(np.square(unif1 - 0.5) + np.square(unif2 - 0.5)) # Distance of each point from the center (0.5, 0.5)
    points_in_circle = np.sum(dist <= 0.5) # total number of points that are inside the circle of radius 0.5
    pi = 4 * np.divide(np.cumsum(dist <= 0.5), indices) # pi is obtained as pi = 4 * (points in circle/total number of points)

    # Obtain cordinates of points that are in and outside the circle
    x_in = unif1[np.where(dist <= 0.5)]
    y_in = unif2[np.where(dist <= 0.5)]
    x_out = unif1[np.where(dist > 0.5)]
    y_out = unif2[np.where(dist > 0.5)]
    
    print(f'Points that fall within the circle = {points_in_circle}')
    print(f'Total number of points = {len(unif1)}')
    print(f'------------------------')
    pi_estimated = 4 * (points_in_circle/len(unif1))
    print(f'Estimation of value of \u03C0 = {pi_estimated}')
    
    trace1 = go.Scatter(x = x_in, y = y_in, mode = 'markers', marker = dict(line = dict(color = 'black', width = 1), color = colorU), name = 'Inside Circle')
    
    trace2 = go.Scatter(x = x_out, y = y_out, mode = 'markers', marker = dict(line = dict(color = 'black', width = 1), color = 'red'), name = 'Outside Circle', opacity = 0.8)
    
    trace3 = go.Scatter(x = indices, y = pi, mode = 'lines', marker = dict(color = colorU), name = 'Estimated pi')
    
    trace4 = go.Scatter(x = indices, y = np.repeat(math.pi, len(indices)), mode = 'lines', marker = dict(color = 'red'), name = 'True pi')
    
    # Create subplots grid of 1 row and 3 columns
    fig = make_subplots(rows = 1, cols = 2, subplot_titles = ["Scatter Plot of Points", "Estimation of π Over Iterations"])
    
    fig.add_trace(trace1, row = 1, col = 1)
    fig.add_trace(trace2, row = 1, col = 1)
    fig.add_trace(trace3, row = 1, col = 2)
    fig.add_trace(trace4, row = 1, col = 2)
    fig.add_shape(type = "circle", xref = "x", yref = "y", x0 = 0, y0 = 0, x1 = 1, y1 = 1, line_color = "black", line_width = 2, row = 1, col = 1)
    fig.update_layout(title = {'text' : f'Monte-Carlo Simulation for estimating the value of π<br><sup>{nameU} Generator</sup><br><sup><sup>(Estimated value of pi = {pi_estimated})</sup>',
                                   'x' : 0.5,
                                   'y' : 0.95,
                                   'xanchor' : 'center',
                                   'yanchor' : 'top',
                                   'font' : {'size' : 25}
                                   },
                        width = 1200,
                        height = 600,
                        paper_bgcolor = 'white',
                        plot_bgcolor = '#777777',
                        legend=dict(bgcolor='lightgray')
                    )
    fig.add_annotation(text = 'pi', xref = 'paper', yref = 'paper', y = math.pi, font=dict(size=15, color="red"), showarrow = False)
    fig.update_xaxes(range = [-0.05, 1.05], title = 'X<sub>1</sub> ~ Unif(0,1)', tickfont_size = 15, title_font_size = 15, scaleanchor = 'y', gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = 1, constrain = 'domain')
    fig.update_yaxes(range = [-0.05, 1.05], title = 'X<sub>2</sub> ~ Unif(0,1)',tickfont_size = 15, title_font_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = 1, constrain = 'domain')
    fig.update_yaxes(range = [min(pi[0:8] - 0.1), max(pi[0:20] + 0.1)], title = 'pi', tickfont_size = 15, title_font_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = 2)
    fig.update_xaxes(range = [-100, n], title = 'Number of data points', tickfont_size = 15, title_font_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = 2)
    fig.show()

In [ ]:
# CELL 44
# To perform the Monte-Carlo simulation for generating the value of pi, we need two sets of random numbers.
# Here we will create another set of 1_000_000 sample from the three generators and then add them into the
# gen_list of dictionary. The purpose is to use the list of dictionary later for Random Variate Generation.

# Tausworthe generator:
t2_unif = Tausworthe({'seed' : '1011001101010100110000110100011101011011101100101110101101011101100101110001011010111100110101111101', 'r' : 53, 'q' : 97, 'l' : 53}).get_unif(n = 1_000_000, disable_tqdm = True, enable_print = False)

# # L'Ecuyer generator:
s21 = [17850, 35123689, 1938350245]
s22 = [115295312, 3126325892, 412866538]
e2_unif = MRG32k3a({'s1' : s21, 's2' :s22}).get_unif(1_000_000, disable_tqdm = True, enable_print = False)

# Mersenne-Twister generator:
mt2_unif = MT19937({'seed' : 2564}).get_unif(1_000_000, disable_tqdm = True, enable_print = False)

# Let's add these three sets of random numbers into the gen_list
add_unif = [t2_unif, e2_unif, mt2_unif]
for i, unif in enumerate(add_unif):
    gen_list[i]['unif2'] = add_unif[i]

### Tausworthe

In [ ]:
# CELL 45
# Now let's supply the Monte-Carlo pi simulation function with the gen_list and generate for the Tausworthe generator first.
# Note that only 10,000 points were used in the plot just for visual clarity and not overwhelm the plot.

MC(gen_list, 'Tausworthe')

### L'Ecuyer

In [ ]:
# CELL 46
# Supply the Monte-Carlo pi simulation function with the gen_list and generate for the L'Ecuyer generator first.
# As before, only 10,000 points were used in the plot just for visual clarity and not overwhelm the plot.

MC(gen_list, 'L\'Ecuyer')

### Mersenne Twister

In [ ]:
# CELL 47
# Finally do the same for Mersenne-Twister generator
# mt1_mc = MT19937({'seed' : 2503}).get_unif(10000, disable_tqdm = True, enable_print = False)
# mt2_mc = MT19937({'seed' : 6298}).get_unif(10000, disable_tqdm = True, enable_print = False)

MC(gen_list, 'Mersenne Twister')

## Random Walk

In [ ]:
# CELL 48
def random_walk(unifs):

    fig = go.Figure()
    # Loop through the three generators
    for unif in unifs:
        # Inside the np.cumsum function, we take each value and assign it -1 if it is less than
        # zero, and +1 if it is greater than or equal to zero. And the cumsum function adds is
        # is just performing a cumulative sum
        walk = np.cumsum(2*(unif['unif'] >= 0.5) - 1)
        trace = go.Scatter(x = np.arange(start = 0, stop = len(unif['unif']), step = 1),
                           y = walk,
                           mode = 'lines',
                           marker = dict(color = unif['color']),
                           name = unif['name'])
        fig.add_trace(trace)
        fig.add_hline(y = 0, line_width = 1, line_color = 'red')
        
    fig.update_xaxes(title_text = 'Step Number', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1)
    fig.update_yaxes(title_text = 'Displacement', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1)
    fig.update_layout(title = {'text' : '<b>Random Walk</b>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'textcase' : 'upper',
                                          'size' : 25}
                                          },
                    legend=dict(bgcolor='lightgray'),
                    width = 1000,
                    height = 600,
                    paper_bgcolor = 'white',
                    plot_bgcolor = '#777777')
    fig.show()

In [ ]:
# CELL 49
# Let's run the random walk function
random_walk(gen_list)

## Random Variate Generation

### Exp(1) via Inverse Transform Theorem

We will use the Inverse Transform Theorem to generate an Exp(1) RV using Unif(0,1).

Let Y ~ Exp(1), with CDF $F(y) = 1 - exp\{-y\}$ then we know that via the Inverse Transform method, if we are given X ~ Unif(0, 1) we can generate Y as follows:
$$
Y = - ln(1 - X)
$$

In [ ]:
# CELL 50
def exp(gen_list, gen):
    for item in gen_list:
        if item['name'] == gen:
            unif = item['unif']
            nameU = item['name']
            colorU = item['color']

    exp_1 = - np.log(1 - unif)
    print(f'----------------------------')
    print(f'Y ~ Exp(1): {exp_1}')
    print(f'----------------------------')
    fig = go.Figure(data = [go.Histogram(x = exp_1, 
                                         xbins = dict(start = min(exp_1), end = max(exp_1), size = 1/5),
                                         marker=dict(line=dict(color='black', width=2), color = colorU),
                                         hovertemplate="<br>".join([f'{nameU}', 'Range: %{x}', 'Count of Random Numbers: %{y}']),
                                         name = '')])
    fig.update_layout(title = {'text' : f'Random Variate Generation Exp(1)<br><sub>{nameU}</sub>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'size' : 25}},
                    width = 800,
                    height = 600,
                    paper_bgcolor = 'white',
                    plot_bgcolor = '#777777')
    fig.update_xaxes(title_text = 'Random Variate, Y', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1)
    fig.update_yaxes(title_text = 'Frequency', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1)
    fig.show()
    
    return exp_1

In [ ]:
# CELL 51
# Let's try out with the samples generated from the Tausworthe generator
t1_exp = exp(gen_list, 'Tausworthe')

In [ ]:
# CELL 52
# Next try the L'Ecuyer generator
e1_exp = exp(gen_list, 'L\'Ecuyer')

In [ ]:
# CELL 53
# Finally, Mersenne Twister
mt1_exp = exp(gen_list, 'Mersenne Twister')

### Tria(0, 1, 2) via Convolution

We will use two Unif(0,1)'s and via Convolution, generate a Tria(0, 1, 2) Random Variate.

In [ ]:
# CELL 54
# Let's generate a Tria(0, 1, 2) distribution from two given Unif(0,1)

def tria(gen_list, gen):
    # Get the distribution
    for item in gen_list:
        if item['name'] == gen:
            unif1 = item['unif']
            unif2 = item['unif2']
            nameU = item['name']
            colorU = item['color']

    # Add the two distributions to generate the triangular distribution
    tria_0_1_2 = np.add(unif1,unif2)
    print(f'----------------------------')
    print(f'Y ~ Tria(0, 1, 2): {tria_0_1_2}')
    print(f'----------------------------')
    fig = go.Figure(data = [go.Histogram(x = tria_0_1_2, 
                                         xbins = dict(start = 0, end = 2, size = 1/20),
                                         marker=dict(line=dict(color='black', width=2), color = colorU),
                                         hovertemplate="<br>".join([f'{nameU}', 'Range: %{x}', 'Count of Random Numbers: %{y}']),
                                         name = '')])
    fig.update_layout(title = {'text' : f'Random Variate Generation Tria(0, 1, 2)<br><sub>{nameU}</sub>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'size' : 25}},
                    width = 800,
                    height = 600,
                    paper_bgcolor = 'white',
                    plot_bgcolor = '#777777')
    fig.update_xaxes(title_text = 'Random Variate, Y', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1)
    fig.update_yaxes(title_text = 'Frequency', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1)
    fig.show()
    
    return tria_0_1_2

In [ ]:
# CELL 55
# Now use the function to generate a Tria(0, 1, 2) distribution using random numbers generated by the Tausworthe generator
t1_tria = tria(gen_list, 'Tausworthe')

In [ ]:
# CELL 56
# We do the same for the L'Ecuyer generator
e1_tria = tria(gen_list, 'L\'Ecuyer')

In [ ]:
# CELL 57
# # And finally for the Mersenne Twister
mt1_tria = tria(gen_list, 'Mersenne Twister')

### Nor(0, 1) via Box-Muller

Let's use Box-Muller method to generate two Nor(0,1) distributions $Z_1$ and $Z_2$ with the help of the two sets of Unif(0, 1)

In [ ]:
# CELL 58
def nor(gen_list, gen):
    # Get the unif distribution
    for item in gen_list:
        if item['name'] == gen:
            unif1 = item['unif']
            unif2 = item['unif2']
            nameU = item['name']
            colorU = item['color']

    # Use the Box-Muller equations to generate Nor
    z1 = np.sqrt(-2 * np.log(unif1)) * np.cos(2 * math.pi * unif2)
    z2 = np.sqrt(-2 * np.log(unif1)) * np.sin(2 * math.pi * unif2)
    print(f'----------------------------')
    print(f'Z\u2081 ~ Nor(0, 1): {z1}')
    print(f'----------------------------')
    print(f'Z\u2082 ~ Nor(0, 1): {z2}')
    print(f'----------------------------')
    
    trace1 = go.Histogram(x = z1, 
                        xbins = dict(start = min(z1), end = max(z1), size = 1/5),
                        marker=dict(line=dict(color = 'black', width = 2), color = colorU),
                        hovertemplate="<br>".join([f'{nameU}', 'Range: %{x}', 'Count of Random Numbers: %{y}']),
                        name = '')
    trace2 = go.Histogram(x = z2, 
                        xbins = dict(start = min(z2), end = max(z2), size = 1/5),
                        marker=dict(line=dict(color = 'black', width = 2), color = colorU),
                        hovertemplate="<br>".join([f'{nameU}', 'Range: %{x}', 'Count of Random Numbers: %{y}']),
                        name = '')
    fig = make_subplots(rows = 1, cols = 2, subplot_titles = ["Distribution of Z<sub>1</sub> ~ Nor(0, 1)", "Distribution of Z<sub>2</sub> ~ Nor(0, 1)"])
    fig.add_trace(trace1, row = 1, col = 1)
    fig.add_trace(trace2, row = 1, col = 2)
    fig.update_layout(title = {'text' : f'Random Variate Generation Nor(0, 1)<br><sub>{nameU}</sub>',
                                'x' : 0.5,
                                'y' : 0.95,
                                'xanchor' : 'center',
                                'yanchor' : 'top',
                                'font' : {'size' : 25}},
                    width = 1200,
                    height = 600,
                    paper_bgcolor = 'white',
                    plot_bgcolor = '#777777')
    fig.update_xaxes(title_text = 'Random Variate, Z<sub>1</sub>', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = 1)
    fig.update_xaxes(title_text = 'Random Variate, Z<sub>2</sub>', title_font_size = 20, tickfont_size = 15, gridwidth = 1, griddash = 'dot', zerolinecolor = 'black', zerolinewidth = 1, row = 1, col = 2)
    fig.update_yaxes(title_text = 'Frequency', title_font_size = 20, gridwidth = 1, griddash = 'dot', tickfont_size = 15, zerolinecolor = 'black', zerolinewidth = 1)
    fig.update_layout(showlegend = False)
    fig.show()
    
    return z1, z2

In [ ]:
# CELL 59
# # Let's try to generate two Nor(0,1) using the Box-Muller method for the Tausworthe generator
t1_nor, t2_nor = nor(gen_list, 'Tausworthe')

In [ ]:
# CELL 60
# # Now let's try the same with the L'Ecuyer generator
e1_nor, e2_nor = nor(gen_list, 'L\'Ecuyer')

In [ ]:
# CELL 61
# # And finally with the Mersenne Twister
mt1_nor, mt2_nor = nor(gen_list, 'Mersenne Twister')